In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import os

2023-09-14 06:27:57.826328: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 06:27:58.650279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load NN model

## Load Data

In [2]:
# Load one video and its csv for testing

path_video = "../data/raw/videos/510A.mp4"
path_csv = "../data/raw/csvs/HD_ChR2_510A.csv"

df = pd.read_csv(os.path.join(path_csv), header=0)
video = cv2.VideoCapture(path_video)


## Process Data

In [3]:
# Function to extract data given a csv file and a video path
def preprocess_video(df, video):
    expansion = 80  # Define the size of the crop

    # From the csv, get the rat's position so we can crop the frames
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    df.set_index(df.columns[0], inplace=True)

    midbody = np.concatenate((df['midbody_y'].values[:, np.newaxis], df['midbody_x'].values[:, np.newaxis]), axis=1)
    midbody = midbody.astype(int)

    # Get video and crop its frames
    success, image = video.read()
    count = 0
    frames = []

    while success:
        # Crop image based on mouse midbody (make boxes having the mouse in the middle, should be square)
        top = max(0, midbody[count][0] - expansion) - max(0, midbody[count][0] + expansion - image.shape[0])
        bottom = min(image.shape[0], midbody[count][0] + expansion) + max(0, expansion - midbody[count][0])
        left = max(0, midbody[count][1] - expansion) - max(0, midbody[count][1] + expansion - image.shape[1])
        right = min(image.shape[1], midbody[count][1] + expansion) + max(0, expansion - midbody[count][1])

        frame = image[top:bottom, left:right]

        # Save frame
        frames.append(frame)

        success, image = video.read()
        count += 1

    return frames

In [4]:
# Get the processed data that will serve as input to the backbone
inputs = preprocess_video(df, video)

## Extract features through backbone model

In [5]:
class FeatureExtractor(tf.keras.Model):
    def __init__(self, model_name):
        """
            Initialize Feature extractor with a pretrained CNN model

            Args:
                model_name: name of the pretrained CNN model ["resnet", "inception_resnet"]
        """
        super(FeatureExtractor, self).__init__()
        if model_name == "resnet":
            from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
            self.model = ResNet50V2(include_top=False, weights='imagenet', pooling='avg')
            self.model_input_size = (224, 224)
        elif model_name == "inception_resnet":
            from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
            self.model = InceptionResNetV2(include_top=False, weights='imagenet', pooling='avg')
            self.model_input_size = (299, 299)
        else:
            raise NameError('Invalid pretrained model name - must be one of ["resnet", "inception_resnet"]')

        self.preprocess_input = preprocess_input
        self.model.trainable = False

    def call(self, inputs):
        """
            Call the pretrained CNN model to predict the features for a given input image

            Args:
                inputs: input image tensor
        """
        # Resize inputs to the expected input size
        inputs = inputs * 255
        inputs = tf.image.resize(inputs, self.model_input_size)
        inputs = inputs[tf.newaxis, :]
        preprocessed_input = self.preprocess_input(inputs)
        return self.model.predict(preprocessed_input).ravel()

In [6]:
# Function to extract features with Resnet backbone model
def feature_extraction(frames):
    features = []
    feature_extractor = FeatureExtractor('resnet')

    for frame in frames:
        #frame = tf.convert_to_tensor(frame)
        #frame = tf.image.decode_image(frame, channels=3)
        #frame = tf.image.convert_image_dtype(frame, tf.float32)
        features.append(feature_extractor(frame))

    return np.array(features)

In [7]:
# Import the NN model
model = tf.keras.models.load_model('resnet_lstm_accuracy.h5')

2023-09-14 06:28:05.225292: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-14 06:28:05.272683: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-14 06:28:05.272755: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-14 06:28:05.274687: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-14 06:28:05.274751: I tensorflow/compile

In [8]:
results = pd.DataFrame()
grooming = []
count = 0  

#for i in range(0,len(inputs),100):
for i in range(0,1200,300):
    print('Batch ', i)
    f = feature_extraction(inputs[i: i+300])
    f2 = np.zeros((1,300,2048))
    f2[0] = f
    result = model.predict(f2)
    result[result>=0.5] = 1
    result[result<0.5] = 0
    grooming += result.tolist() 

Batch  0


2023-09-14 06:28:10.482102: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 17ms/step


2023-09-14 06:28:32.942728: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-14 06:28:32.945204: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-14 06:28:32.946474: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

1/1 [==============================] - 0s 33ms/step
Batch  600
1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 30ms/step
Batch  900
1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - ETA: 0s

In [13]:
grooming

In [14]:
results['grooming'] = grooming[0][:]

In [26]:
results.head

<bound method NDFrame.head of     grooming
0      [0.0]
1      [0.0]
2      [0.0]
3      [0.0]
4      [0.0]
..       ...
295    [0.0]
296    [0.0]
297    [0.0]
298    [0.0]
299    [0.0]

[300 rows x 1 columns]>